In [153]:
# Import the os package
import os

# Import the openai package
import openai
from openai import OpenAI

# From the IPython.display package, import display and Markdown
from IPython.display import display, Markdown

# Use the dotenv package
from dotenv import load_dotenv

# Use the CSV Reader Package
import csv

# Use regular expressions
import re

# Use subprocess
import subprocess

load_dotenv()  # This loads the .env file at the root of the project

# Set openai.api_key to the OPENAI environment variable
client = OpenAI(
    api_key = os.getenv("OPENAI")
)
# print(os.getenv("OPENAI"))

In [154]:
def chat(client, system, user_assistant, openai_model):
    """
    A simple function to handle the mechanics of chatting with ChatGPT
    The model is passed in to make it easy to switch models for different tasks
    """
    assert isinstance(system, str), "`system` should be a string"
    assert isinstance(user_assistant, list), "`user_assistant` should be a list"
    system_msg = [{"role": "system", "content": system}]
    # user_assistant_msgs = [
    #     {"role": "assistant", "content": user_assistant[i]} if i % 2 else {"role": "user", "content": user_assistant[i]}
    #     for i in range(len(user_assistant))]
    user_assistant_msgs = []
    for item in user_assistant:
        content = item['contents']
        msg = {"role": "user", "content": content}
        user_assistant_msgs.append(msg)
    msgs = system_msg + user_assistant_msgs
    response = client.chat.completions.create(model=openai_model, messages=msgs)
    status_code = response.choices[0].finish_reason
    assert status_code == "stop", f"The status code was {status_code}."
    # print(f'{response=}')
    return response.choices[0].message.content

In [155]:
def read_csv_file(file_path):
    """
    Reads a CSV file with each row containing a title and a script path.
    Returns a list of dictionaries with keys 'title' and 'code_path'.
    """
    data = []
    with open(file_path, mode='r', encoding='utf-8') as file:
        reader = csv.reader(file)
        data.extend(
            {'title': row[0], 'code_path': row[1]}
            for row in reader
            if len(row) == 2
        )
    return data


In [156]:
def read_files_and_update_csv_data(csv_data, repo_root):
    """
    Reads files specified in the data structure and concatenates their contents.
    
    Parameters:
    - csv_data (list of dict): List of dictionaries with 'title' and 'code_path'.
    - repo_root (str): Repository root base location.

    Returns:
    - str: A single string containing all the titles and file contents.
    """
    csv_data_contents = []
    for item in csv_data:
        full_path = os.path.join(repo_root, item['code_path'])
        # print(f'{full_path=}')
        full_path = full_path.replace(' ./','/')
        try:
            with open(full_path, 'r', encoding='utf-8') as file:
                contents = file.read()
                item['contents'] = contents
                csv_data_contents.append(item)
        except FileNotFoundError:
            concatenated_contents += f"{item['title']}:\n<File not found: {full_path}>\n\n\n"
        except Exception as e:
            concatenated_contents += f"{item['title']}:\n<Error reading file: {e}>\n\n\n"

    return csv_data_contents


In [157]:
def find_repo_root(start_path):
    return subprocess.check_output(['git', 'rev-parse', '--show-toplevel']).strip().decode('utf-8')

In [158]:
def save_markdown_file(item, repo_root, path, response_in):
    title = item['title']
    file_path = os.path.join(repo_root, path, f"{title}.md")

    # Extract the content for each block
    content = response_in

    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(content.strip() + '\n')

In [159]:
# Read the CSV File
file_path = '/home/sean/github/RTLDesignSherpa/scripts.csv'
csv_data = read_csv_file(file_path)
print(csv_data)

# Concatenate the Files
repo_root = find_repo_root(os.getcwd())
print(repo_root)
csv_data_contents = read_files_and_update_csv_data(csv_data, repo_root)
# for item in csv_data_contents:
#     print(f'{item["title"]}:{item["code_path"]}\n{item["contents"]}\n\n\n')

[{'title': 'lint', 'code_path': ' ./bin/project_automation/lint.py'}, {'title': 'run_test', 'code_path': ' ./bin/project_automation/run_test.py'}, {'title': 'list_test', 'code_path': ' ./bin/project_automation/list_test.py'}, {'title': 'REMatcher', 'code_path': ' ./bin/project_automation/REMatcher.py'}, {'title': 'verilog_parser', 'code_path': ' ./bin/rtl_generators/verilog/verilog_parser.py'}, {'title': 'module', 'code_path': ' ./bin/rtl_generators/verilog/module.py'}, {'title': 'param', 'code_path': ' ./bin/rtl_generators/verilog/param.py'}, {'title': 'signal', 'code_path': ' ./bin/rtl_generators/verilog/signal.py'}, {'title': 'wallace_multiplier', 'code_path': ' ./bin/rtl_generators/multipliers/generates_slow_sims/wallace_multiplier.py'}, {'title': 'dadda_reduction', 'code_path': ' ./bin/rtl_generators/multipliers/generates_slow_sims/dadda_reduction.py'}, {'title': 'wallace_reduction', 'code_path': ' ./bin/rtl_generators/multipliers/generates_slow_sims/wallace_reduction.py'}, {'titl

In [160]:
openai_model = "gpt-4-1106-preview"
my_header_prompt = '''
In the following text, there will be sections of code that have a header such as:
<block>: <path to block>
followed by a block of code

Please give the Mark Down for the block providing full documentation of the code with knowledge of the other code that is referenced or used as
an include file. The Mark Down for each code block should start with a H1 <block> to indicate when a new block begins.
It should also include full details of the inputs/outputs and internal functionality along with any command line options needed.
At the end of a block, there should be two new lines to provide a visual indicator also.

'''
for item in csv_data_contents:
    prompt = f'{my_header_prompt}{item["title"]}: {item["code_path"]}\n{item["contents"]}\n\n'
    # print(prompt)
    item['contents'] = prompt
    response_fn_test = chat(client, "You are an expert in python and verilog.", [item], openai_model)
    save_markdown_file(item, repo_root, 'docs/mark_down/scripts/', response_fn_test)


response=ChatCompletion(id='chatcmpl-8YoRkeT15YsrVrVfiqm498TrP1cLF', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content="# lint\n\n## Overview\nThe `lint` class in `./bin/project_automation/lint.py` is designed to perform linting and formatting on RTL (Register Transfer Level) code, specifically Verilog files, using a variety of tools such as Verible and Yosys. The class ensures that the code follows certain style guidelines and checks for typical coding mistakes.\n\n## Class: Lint\n\n### Description\nThe `Lint` class provides methods to run two main operations:\n\n- Formatting Verilog files using Verible.\n- Linting Verilog files using Verible's linter and Yosys.\n\nThe class maintains the directory structure for reports and uses a configuration file to manage the linting process.\n\n### Attributes\n- `original_directory (str)`: Stores the current working directory at the time the class is instantiated.\n- `repo_root (str)`: The absolute path to the r

In [161]:
# save_markdown_files(csv_data, repo_root, 'docs/mark_down/scripts/', response_fn_test)